In [1]:
import pandas as pd
import numpy as np
import json as js
from datetime import datetime
from dateutil.relativedelta import relativedelta

#read in csv and/or dataset
water_df = pd.read_csv('../SourceMaterial/Measures/water_temps.csv')
ph_df = pd.read_csv('../SourceMaterial/Measures/ph.csv')
conductivity_df = pd.read_csv('../SourceMaterial/Measures/conductivity.csv')
water_df = water_df.dropna()
ph_df = ph_df.dropna()
conductivity_df = conductivity_df.dropna()
df_list = [water_df, ph_df, conductivity_df]
#order by HUC then Date
for dframe in df_list:
    dframe.sort_values(by=['HUCNAME_', 'Date'])


results = []
result_df_list = []
result_dict = {}
key_list = ['water', 'ph', 'conductivity']

C:\Users\jacob\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
def checkdates(working_df, huc):
    df = working_df.loc[working_df['HUCNAME_']==huc]
    df['datetime'] = [datetime.strptime(value, '%m/%d/%Y') for value in df['Date']]
    df = df.sort_values(by=['datetime'])
    df['datediff'] = df['datetime'].diff()
    datetimecol = df['datetime']
    min_date = min(datetimecol)
    max_date = max(datetimecol)
    diff_years = relativedelta(max_date, min_date).years
    df = df.dropna()
    datedifcol = df['datediff'].dt.days
    try:
        max_diff = max(datedifcol)
        if diff_years >=3 and max_diff <= 100:
            datecheck = True
        else:
            datecheck = False
    except:
        datecheck = False
        max_diff = None
        
    return datecheck, max_diff, max_date, min_date

In [3]:
def split_quarters(df, huc):
    #for each suitable HUC, split temps into quarters by date
    df['month'] = [int(value[0:1]) for value in df['Date']]
    df['year'] = [value[6:10] for value in df['Date']]
    df['q'] = ''
        
    df['q'].loc[df['month'] <=3] = 'Q1'
    df['q'].loc[df['month'] >=4] = 'Q2'
    df['q'].loc[df['month'] >=7] = 'Q3'
    df['q'].loc[df['month'] >=10] = 'Q4'
        
    df['Quarter'] = df['q'] + " "+ df['year']
        
    df.drop(columns = ['month', 'year'])
        
    #calculate variance or stdev of temps and avg temps for each quarter
    quarters_list = df['Quarter'].unique()
    return df, quarters_list
    

In [4]:
def analyze_df(df, quarters_list, huc):
    for quarter in quarters_list:
            quarter_df = df.loc[df['Quarter']==quarter]
            values = [value for value in quarter_df['MeasureValue']]
            variance = np.var(values)
            stdev = np.std(values)
            average = np.average(values)
            max_val = max(values)
            min_val = min(values)
            result = {
                'huc':huc, 
                'quarter': quarter, 
                'var' : variance, 
                'stdev' : stdev, 
                'avg' : average, 
                'max' : max_val, 
                'min' : min_val,
                'max_time_diff': max_diff,
                'max_date': max_date,
                'min_date': min_date
            }
    return result

In [5]:
for dframe in df_list:
    hucs = dframe['HUCNAME_'].unique()
    for huc in hucs:
        datecheck, max_diff, max_date, min_date = checkdates(dframe, huc)
        if datecheck == True:
            df_quarters, quarters_list = split_quarters(dframe, huc)
            result = analyze_df(df_quarters, quarters_list, huc)
            results.append(result)
        else:
            pass
    result_df = pd.DataFrame(results)
    result_df_list.append(result_df)
for key in key_list:
    result_dict[key_list[key_list.index(key)]] = result_df_list[key_list.index(key)]
    result_dict[key_list[key_list.index(key)]].to_csv(f'../SourceMaterial/Measures/Results/{key}_result.csv')        
    

C:\Users\jacob\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jacob\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
